In [31]:
import sys
import os
import geopandas as gpd
import pandas as pd

# Add the parent directory of 'src' to sys.path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from src.calculate import calc_leistungskm
from src.import_gpx import import_gpx

from src.maps import generate_elevation_plot 
from src.MICHI import export_to_pdf


In [32]:
gpx = import_gpx("C:/Users/aebim/Documents/02_Ausbildung/Studium/04_Semester/4040_Geoprogrammieren_II/GrSchmeterling_Hackathon/data/MeileBH_Buech.gpx")

c:\Users\aebim\anaconda3\envs\4040_Hackaton\lib\site-packages\geopandas\io\file.py:383: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  as_dt = pd.to_datetime(df[k], errors="ignore")
c:\Users\aebim\anaconda3\envs\4040_Hackaton\lib\site-packages\geopandas\geoseries.py:645: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  result = super().apply(func, convert_dtype=convert_dtype, args=args, **kwargs)
c:\Users\aebim\anaconda3\envs\4040_Hackaton\lib\site-packages\geopandas\geoseries.py:645: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  result = super().apply(func, convert_dtype=convert_dtype, args=args, **kwargs)
c:\

In [29]:
gpx.head()


,segment_id,von_pkt_name,von_pkt_geom,bis_pkt_name,bis_pkt_geom,segment_geom
0,1,Bahnhof,POINT Z (2691201.326 1236128.273 419.800),Burg,POINT Z (2691048.922 1237146.257 501.600),"LINESTRING Z (2691201.326 1236128.273 419.800,..."
1,2,Burg,POINT Z (2691048.922 1237146.257 501.600),Übereschüttle,POINT Z (2691361.068 1238027.360 609.800),"LINESTRING Z (2691048.922 1237146.257 501.600,..."
2,3,Übereschüttle,POINT Z (2691361.068 1238027.360 609.800),Ratsfelse,POINT Z (2691110.075 1238241.493 614.200),"LINESTRING Z (2691361.068 1238027.360 609.800,..."
3,4,Ratsfelse,POINT Z (2691110.075 1238241.493 614.200),Buech,POINT Z (2690088.967 1238277.303 584.600),"LINESTRING Z (2691110.075 1238241.493 614.200,..."


In [30]:
gdf_calc = calc_leistungskm(gpx)

TypeError: unsupported operand type(s) for /: 'list' and 'int'

In [ ]:
print(gdf_calc)
print(type(gdf_calc))


(   segment_id   von_pkt_name                               von_pkt_geom  \
0           1        Bahnhof  POINT Z (2691201.326 1236128.273 419.800)   
1           2           Burg  POINT Z (2691048.922 1237146.257 501.600)   
2           3  Übereschüttle  POINT Z (2691361.068 1238027.360 609.800)   
3           4      Ratsfelse  POINT Z (2691110.075 1238241.493 614.200)   

    bis_pkt_name                               bis_pkt_geom  \
0           Burg  POINT Z (2691048.922 1237146.257 501.600)   
1  Übereschüttle  POINT Z (2691361.068 1238027.360 609.800)   
2      Ratsfelse  POINT Z (2691110.075 1238241.493 614.200)   
3          Buech  POINT Z (2690088.967 1238277.303 584.600)   

                                        segment_geom  cumulative_km  \
0  LINESTRING Z (2691201.326 1236128.273 419.800,...       1.610076   
1  LINESTRING Z (2691048.922 1237146.257 501.600,...       1.140369   
2  LINESTRING Z (2691361.068 1238027.360 609.800,...       0.652464   
3  LINESTRING Z (269111

In [ ]:
df = gdf_calc[0]  # get the DataFrame from the tuple

gdf_show = pd.DataFrame({
    'Abschnitt': df['segment_id'],
    'Von': df['von_pkt_name'],
    'Nach': df['bis_pkt_name'],
    'Laenge [km]': df['cumulative_km'].round(2),
    'Hoehenmeter [m]': df['elevation'].round(1),
    'Leistungskm': df['Leistungskm [km]'].round(2),
    'Marschzeit [min]': df['Marschzeit [min]'].round(0),
    'Hinweis': ""  # Add an empty 'Hinweis' column
})


gdf_show.head()


,Abschnitt,Von,Nach,Laenge [km],Hoehenmeter [m],Leistungskm,Marschzeit [min],Hinweis
0,1,Bahnhof,Burg,1.61,81.8,3.27,49.0,
1,2,Burg,Übereschüttle,1.14,108.2,2.39,36.0,
2,3,Übereschüttle,Ratsfelse,0.65,4.4,1.50,22.0,
3,4,Ratsfelse,Buech,1.32,-29.6,2.22,33.0,


In [34]:
import geopandas as gpd
from shapely.geometry import LineString
import numpy as np
import pandas as pd

# Import your GPX data
gdf = import_gpx("C:/Users/aebim/Documents/02_Ausbildung/Studium/04_Semester/4040_Geoprogrammieren_II/GrSchmeterling_Hackathon/data/MeileBH_Buech.gpx")
pace = 4.0  # km/h

# Prepare new columns
gdf['cumulative_km'] = 0.0
gdf['elevation'] = 0.0
gdf['Leistungskm [km]'] = 0.0
gdf['Marschzeit [min]'] = 0.0

# Loop and process each segment
for idx, row in gdf.iterrows():
    line = row.segment_geom
    coords = np.array(line.coords)

    # Calculate horizontal distance
    delta_xy = np.diff(coords[:, :2], axis=0)
    d_segs = np.linalg.norm(delta_xy, axis=1)  # meters
    km = np.sum(d_segs) / 1000.0  # km

    # Elevation change
    delta_z = np.diff(coords[:, 2])
    h_segs = delta_z.copy()
    hm = np.sum(h_segs)  # total elevation change

    # Effort-adjusted distance
    leistungskm_segs = d_segs / 1000.0 + np.abs(h_segs) / 100.0
    leistungskm = np.sum(leistungskm_segs)

    # March time in minutes
    mz = round((leistungskm / pace) * 60, 0)

    # Store in DataFrame
    gdf.at[idx, 'cumulative_km'] = km
    gdf.at[idx, 'elevation'] = hm
    gdf.at[idx, 'Leistungskm [km]'] = leistungskm
    gdf.at[idx, 'Marschzeit [min]'] = mz

    # After loop: compute totals
    km_list = gdf['cumulative_km'].tolist()
    hm_list = gdf['elevation'].tolist()
    leistungskm_list = gdf['Leistungskm [km]'].tolist()
    mz_list = gdf['Marschzeit [min]'].tolist()

    # Recalculate elevation segments
    hm_array = np.array(hm_list)
    h_segs = np.diff(hm_array)

tot_dist = round(np.sum(km_list), 3)
tot_lkm = round(np.sum(leistungskm_list), 3)
tot_hm_pos = round(np.sum(h_segs[h_segs > 0]), 0)
tot_hm_neg = round(np.sum(h_segs[h_segs < 0]), 0)
tot_marschzeit_h = int(np.sum(mz_list) // 60)
tot_marschzeit_min = int(round(np.sum(mz_list) % 60))

# Output
gdf, tot_dist, tot_lkm, tot_hm_pos, tot_hm_neg, tot_marschzeit_h, tot_marschzeit_min


c:\Users\aebim\anaconda3\envs\4040_Hackaton\lib\site-packages\geopandas\io\file.py:383: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  as_dt = pd.to_datetime(df[k], errors="ignore")
c:\Users\aebim\anaconda3\envs\4040_Hackaton\lib\site-packages\geopandas\geoseries.py:645: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  result = super().apply(func, convert_dtype=convert_dtype, args=args, **kwargs)
c:\Users\aebim\anaconda3\envs\4040_Hackaton\lib\site-packages\geopandas\geoseries.py:645: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  result = super().apply(func, convert_dtype=convert_dtype, args=args, **kwargs)
c:\

(   segment_id   von_pkt_name                               von_pkt_geom  \
 0           1        Bahnhof  POINT Z (2691201.326 1236128.273 419.800)   
 1           2           Burg  POINT Z (2691048.922 1237146.257 501.600)   
 2           3  Übereschüttle  POINT Z (2691361.068 1238027.360 609.800)   
 3           4      Ratsfelse  POINT Z (2691110.075 1238241.493 614.200)   
 
     bis_pkt_name                               bis_pkt_geom  \
 0           Burg  POINT Z (2691048.922 1237146.257 501.600)   
 1  Übereschüttle  POINT Z (2691361.068 1238027.360 609.800)   
 2      Ratsfelse  POINT Z (2691110.075 1238241.493 614.200)   
 3          Buech  POINT Z (2690088.967 1238277.303 584.600)   
 
                                         segment_geom  cumulative_km  \
 0  LINESTRING Z (2691201.326 1236128.273 419.800,...       1.610076   
 1  LINESTRING Z (2691048.922 1237146.257 501.600,...       1.140369   
 2  LINESTRING Z (2691361.068 1238027.360 609.800,...       0.652464   
 3  LINE